In [2]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
import pandas as pd
import ast
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Load and preprocess dataset
file_path = 'NER_Dataset.csv'
data = pd.read_csv(file_path)

def convert_string_to_list(row):
    row['Word'] = ast.literal_eval(row['Word'])
    row['POS'] = ast.literal_eval(row['POS'])
    row['Tag'] = ast.literal_eval(row['Tag'])
    return row

data = data.apply(convert_string_to_list, axis=1)

# Extract unique words and tags
unique_words = set()
unique_tags = set()
for _, row in data.iterrows():
    unique_words.update(row['Word'])
    unique_tags.update(row['Tag'])

word_to_ix = {word: i for i, word in enumerate(unique_words, start=1)} # Reserve 0 for padding
word_to_ix['<UNK>'] = 0  # Unknown words
tag_to_ix = {tag: i for i, tag in enumerate(unique_tags)}

if 'O' not in tag_to_ix:
    tag_to_ix['O'] = len(tag_to_ix)

ix_to_tag = {ix: tag for tag, ix in tag_to_ix.items()}

class NERDataset(Dataset):
    def __init__(self, sentences, tags, word_to_ix, tag_to_ix):
        self.sentences = [[word_to_ix.get(word, 0) for word in sentence] for sentence in sentences]
        self.tags = [[tag_to_ix[tag] for tag in tag_seq] for tag_seq in tags]

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return torch.tensor(self.sentences[idx], dtype=torch.long), torch.tensor(self.tags[idx], dtype=torch.long)

def pad_collate(batch):
    (sentences, tags) = zip(*batch)
    sentences_padded = pad_sequence(sentences, batch_first=True, padding_value=0)
    tags_padded = pad_sequence(tags, batch_first=True, padding_value=tag_to_ix['O'])
    return sentences_padded, tags_padded

class BiLSTMForNER(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim=64, hidden_dim=128):
        super(BiLSTMForNER, self).__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence[0]), sentence.size(0), -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence[0]), -1, lstm_out.shape[-1]))
        tag_scores = torch.log_softmax(tag_space, dim=2)
        return tag_scores

# Prepare data for training and evaluation
sentences = [row['Word'] for _, row in data.iterrows()]
tags = [row['Tag'] for _, row in data.iterrows()]

train_sentences, test_sentences, train_tags, test_tags = train_test_split(sentences, tags, test_size=0.2, random_state=42)

train_dataset = NERDataset(train_sentences, train_tags, word_to_ix, tag_to_ix)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate)

test_dataset = NERDataset(test_sentences, test_tags, word_to_ix, tag_to_ix)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)

# Model, optimizer, and loss function
model = BiLSTMForNER(len(word_to_ix), len(tag_to_ix))
optimizer = Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

# Function to evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    true_tags, pred_tags = [], []

    with torch.no_grad():
        for sentences, tags in data_loader:
            tag_scores = model(sentences)
            predictions = torch.argmax(tag_scores, dim=2)
            true_tags.extend(tags.view(-1).tolist())
            pred_tags.extend(predictions.view(-1).tolist())
    
    true_tags = [ix_to_tag[ix] for ix in true_tags if ix in ix_to_tag]
    pred_tags = [ix_to_tag[ix] for ix in pred_tags if ix in ix_to_tag]

    accuracy = accuracy_score(true_tags, pred_tags)
    report = classification_report(true_tags, pred_tags, labels=list(tag_to_ix.values()), target_names=list(tag_to_ix.keys()))

    return accuracy, report

# Training and evaluation loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for sentences, tags in train_loader:
        model.zero_grad()
        tag_scores = model(sentences)
        loss = loss_function(tag_scores.view(-1, len(tag_to_ix)), tags.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    accuracy, report = evaluate_model(model, test_loader)
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}, Accuracy: {accuracy}")
    


Epoch 1, Loss: 0.30096469200010395, Accuracy: 0.9552918958863618
Epoch 2, Loss: 0.15278345254172973, Accuracy: 0.9625251457426718
Epoch 3, Loss: 0.12434662071936721, Accuracy: 0.9654784875605551
Epoch 4, Loss: 0.11009556925960438, Accuracy: 0.9669384801707858
Epoch 5, Loss: 0.10101765807441515, Accuracy: 0.9677236431562526
Epoch 6, Loss: 0.09465036939851337, Accuracy: 0.9681547130306265
Epoch 7, Loss: 0.09018030775566464, Accuracy: 0.9687397364315625
Epoch 8, Loss: 0.08600317636928874, Accuracy: 0.9687448682157812
Epoch 9, Loss: 0.08382046884787987, Accuracy: 0.9689757985056244
Epoch 10, Loss: 0.0810705796256972, Accuracy: 0.9692041629033582


In [26]:
input_text = 'Prime Minister'

# Tokenize the input text (simple whitespace tokenization for this example)
tokens = input_text.split()

# Convert tokens to their respective indices
token_indices = [word_to_ix.get(token, 0) for token in tokens]  # Use 0 for unknown words ('<UNK>')

# Convert to a tensor and add an extra dimension (batch size of 1)
token_tensor = torch.tensor([token_indices], dtype=torch.long)

model.eval()  # Set the model to evaluation mode

with torch.no_grad():  # No need to track gradients
    predictions = model(token_tensor)  # Predict
    predicted_tags_indices = torch.argmax(predictions, dim=2)  # Get the index of the max log-probability
    predicted_tags = [list(tag_to_ix.keys())[predicted_tag] for predicted_tag in predicted_tags_indices[0]]  # Convert indices to tags

for token, tag in zip(tokens, predicted_tags):
    print(f'{token}: {tag}')


Prime: B-per
